In [7]:
import pandas as pd
import requests
import os 

In [8]:
df10 = pd.read_csv('books10.csv')

In [9]:
system_prompt = """
    You are a helpful literary assistant, helping me with my master thesis on gender in literature. 
    Your job is to search the internet and assess whether a book features a **female major character**.
    
    You always return only a pandas data frame with the columns 'title', 'author', 'female_major_character' (0 = no, 1 = yes),
    and 'expl' (reasoning of why there is or isn't a female major character).

    The female major character does not need to be the protagonist, but she must be a central presence in the narrative, 
    with more significance than a secondary or background character.

    A female character is considered a **major character** if she plays a substantial role in the story, 
    regularly appears throughout the book, and if her role significantly contributes to relationships or events that 
    shape the plot or other characters.

    Characters who serve only as symbolic figures, plot devices, or one-time motivators without meaningful presence 
    or development should not be considered major characters.

    Examples of female major characters:
    - **Katniss Everdeen** in **The Hunger Games** – protagonist, central to the plot, her choices drive the rebellion and story progression.
    – **Hermione Granger** in *Harry Potter* – not the protagonist, but central to plot progression and character development.
    
    Examples of female NON-major characters:
    - **Prim Everdeen** in **The Hunger Games** – important to Katniss's emotional motivation, but doesn’t influence the plot or have significant character development beyond being a catalyst.
    - **Professor McGonagall** in **Harry Potter** – important supporting character, but doesn’t influence the main plot or have significant character development beyond being a mentor and guide to Harry and his friends.
    """

user_prompt_template = "Is there a female main character in the book {title} by {author}?"

nur ja oder nein
dieses Format: ja, Name 

In [57]:
# Find 20 novels, check by hand

# Original data with an extra column
books = [
    {"Title": "Pride and Prejudice", "Author": "Jane Austen", "Female_Major_Character": 1},
    {"Title": "To Kill a Mockingbird", "Author": "Harper Lee", "Female_Major_Character": 1},  # Scout!
    {"Title": "The Book Thief", "Author": "Markus Zusak", "Female_Major_Character": 1},
    {"Title": "Twilight", "Author": "Stephenie Meyer", "Female_Major_Character": 1},
    {"Title": "The Fault in Our Stars", "Author": "John Green", "Female_Major_Character": 1},
    {"Title": "The Perks of Being a Wallflower", "Author": "Stephen Chbosky", "Female_Major_Character": 1},  # Sam!
    {"Title": "Brave New World", "Author": "Aldous Huxley", "Female_Major_Character": 1},  # Lenina is significant
    {"Title": "A Game of Thrones (A Song of Ice and Fire, #1)", "Author": "George R.R. Martin", "Female_Major_Character": 1},  # So many iconic women!
    {"Title": "The Lightning Thief", "Author": "Rick Riordan", "Female_Major_Character": 1},  # Annabeth 💪
    {"Title": "Lolita", "Author": "Vladimir Nabokov", "Female_Major_Character": 1},  # Dolores (Lolita) is central but controversial

    {"Title": "The Hobbit and The Lord of the Rings", "Author": "J.R.R. Tolkien", "Female_Major_Character": 0},
    {"Title": "The Little Prince", "Author": "Antoine de Saint-Exupéry", "Female_Major_Character": 0},
    {"Title": "Crime and Punishment", "Author": "Fyodor Dostoevsky", "Female_Major_Character": 0},  # Some female characters but arguably not "major"
    {"Title": "Lord of the Flies", "Author": "William Golding", "Female_Major_Character": 0},  # All boys 😶‍🌫️
    {"Title": "The Adventures of Huckleberry Finn", "Author": "Mark Twain", "Female_Major_Character": 0},  # Male-centric
    {"Title": "The Old Man and the Sea", "Author": "Ernest Hemingway", "Female_Major_Character": 0},  # No female characters really
    {"Title": "Alex Rider", "Author": "Anthony Horowitz", "Female_Major_Character": 0},  # No female characters really
    {"Title": "Der Trafikant", "Author": "Robert Seethaler", "Female_Major_Character": 0},  # No female characters really
]

# Create the DataFrame
df_books = pd.DataFrame(books)

# Display the updated DataFrame
print(df_books)

                                             Title                    Author  \
0                              Pride and Prejudice               Jane Austen   
1                            To Kill a Mockingbird                Harper Lee   
2                                   The Book Thief              Markus Zusak   
3                                         Twilight           Stephenie Meyer   
4                           The Fault in Our Stars                John Green   
5                  The Perks of Being a Wallflower           Stephen Chbosky   
6                                  Brave New World             Aldous Huxley   
7   A Game of Thrones (A Song of Ice and Fire, #1)        George R.R. Martin   
8                              The Lightning Thief              Rick Riordan   
9                                           Lolita          Vladimir Nabokov   
10            The Hobbit and The Lord of the Rings            J.R.R. Tolkien   
11                               The Lit

In [ ]:
# set tempderature to 0? 

1. Run and look at overlap between Female_Major_Character and female_major_character.
2. Run again and see if it changed. 
3. Run again. 
4. If changes, set temperature to 0.

In [14]:
print(df_books['Title'].head)

<bound method NDFrame.head of 0                                   The Hunger Games
1          Harry Potter and the Order of the Phoenix
2                                Pride and Prejudice
3                              To Kill a Mockingbird
4                                     The Book Thief
5                                           Twilight
6                             The Fault in Our Stars
7                    The Perks of Being a Wallflower
8                                  The Little Prince
9                               Crime and Punishment
10                                 Lord of the Flies
11                                   Brave New World
12    A Game of Thrones (A Song of Ice and Fire, #1)
13                         The Giver (The Giver, #1)
14                The Adventures of Huckleberry Finn
15                                            Lolita
16                           The Old Man and the Sea
Name: Title, dtype: object>


In [15]:
# Setting my API key
YOUR_API_KEY = os.environ["PERPLEXITY_API_KEY"]

# Saying hi it's me, and this is what I'll be sending
headers = {
    "Authorization": f"Bearer {YOUR_API_KEY}",
    "Content-Type": "application/json"
}

# This is where I want to send it to
url = "https://api.perplexity.ai/chat/completions"

# I'll store the responses I get in this (as of now, empty) list
responses = []

# Setting up what I want to send to Perplexity
for index, row in df_books.iterrows():
    # Extract title and author from each row 
    title = row["Title"]
    author = row["Author"]

    # Format user prompt with current title and author
    user_prompt = user_prompt_template.format(title=title, author=author)

    # Payload
    payload = {
        "model": "sonar",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        # "max_tokens": 123,  # Limits the length of the response
        # "temperature": 0.2,  # Controls randomness in the response
        # "top_p": 0.9,
        # "search_domain_filter": ["<any>"],
        # "return_images": False,
        # "return_related_questions": False,
        # "search_recency_filter": "<string>",
        # "top_k": 0,
        # "stream": False,
        # "presence_penalty": 0,
        # "frequency_penalty": 1,
        # "response_format": {},
        # "web_search_options": {"search_context_size": "high"}
    }

    # Send the request to the Perplexity API
    response = requests.post(url, json=payload, headers=headers)
    
    # Check the response
    if response.status_code == 200:
        response_json = response.json()
        # Extract the answer from the response (this might vary depending on the response format)
        answer = response_json.get('choices', [{}])[0].get('message', {}).get('content', 'No answer found')
        responses.append(answer)  # Store the answer
    else:
        # Handle errors by appending an error message
        responses.append(f"Error: {response.status_code}")

# Add the responses to your original dataframe
df_books['test'] = responses

# Print the updated dataframe to check the results
print(df_books)

                                             Title                    Author  \
0                                 The Hunger Games           Suzanne Collins   
1        Harry Potter and the Order of the Phoenix              J.K. Rowling   
2                              Pride and Prejudice               Jane Austen   
3                            To Kill a Mockingbird                Harper Lee   
4                                   The Book Thief              Markus Zusak   
5                                         Twilight           Stephenie Meyer   
6                           The Fault in Our Stars                John Green   
7                  The Perks of Being a Wallflower           Stephen Chbosky   
8                                The Little Prince  Antoine de Saint-Exupéry   
9                             Crime and Punishment         Fyodor Dostoevsky   
10                               Lord of the Flies           William Golding   
11                                 Brave

In [35]:
df_books['test'].iloc[17]

IndexError: single positional indexer is out-of-bounds

Abweichungen: 
    {"Title": "Crime and Punishment", "Author": "Fyodor Dostoevsky", "Female_Major_Character": 0},  # Some female characters but arguably not "major"
    {"Title": "The Giver (The Giver, #1)", "Author": "Lois Lowry", "Female_Major_Character": 1},  # Fiona, the mom, etc.


In [36]:
# Run again: 
# Setting my API key
YOUR_API_KEY = os.environ["PERPLEXITY_API_KEY"]

# Saying hi it's me, and this is what I'll be sending
headers = {
    "Authorization": f"Bearer {YOUR_API_KEY}",
    "Content-Type": "application/json"
}

# This is where I want to send it to
url = "https://api.perplexity.ai/chat/completions"

# I'll store the responses I get in this (as of now, empty) list
responses2 = []

# Setting up what I want to send to Perplexity
for index, row in df_books.iterrows():
    # Extract title and author from each row 
    title = row["Title"]
    author = row["Author"]

    # Format user prompt with current title and author
    user_prompt = user_prompt_template.format(title=title, author=author)

    # Payload
    payload = {
        "model": "sonar",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        # "max_tokens": 123,  # Limits the length of the response
        # "temperature": 0.2,  # Controls randomness in the response
        # "top_p": 0.9,
        # "search_domain_filter": ["<any>"],
        # "return_images": False,
        # "return_related_questions": False,
        # "search_recency_filter": "<string>",
        # "top_k": 0,
        # "stream": False,
        # "presence_penalty": 0,
        # "frequency_penalty": 1,
        # "response_format": {},
        # "web_search_options": {"search_context_size": "high"}
    }

    # Send the request to the Perplexity API
    response = requests.post(url, json=payload, headers=headers)
    
    # Check the response
    if response.status_code == 200:
        response_json = response.json()
        # Extract the answer from the response (this might vary depending on the response format)
        answer = response_json.get('choices', [{}])[0].get('message', {}).get('content', 'No answer found')
        responses2.append(answer)  # Store the answer
    else:
        # Handle errors by appending an error message
        responses2.append(f"Error: {response.status_code}")

# Add the responses to your original dataframe
df_books['test2'] = responses2

# Print the updated dataframe to check the results
print(df_books)

                                             Title                    Author  \
0                                 The Hunger Games           Suzanne Collins   
1        Harry Potter and the Order of the Phoenix              J.K. Rowling   
2                              Pride and Prejudice               Jane Austen   
3                            To Kill a Mockingbird                Harper Lee   
4                                   The Book Thief              Markus Zusak   
5                                         Twilight           Stephenie Meyer   
6                           The Fault in Our Stars                John Green   
7                  The Perks of Being a Wallflower           Stephen Chbosky   
8                                The Little Prince  Antoine de Saint-Exupéry   
9                             Crime and Punishment         Fyodor Dostoevsky   
10                               Lord of the Flies           William Golding   
11                                 Brave

In [56]:
df_books['test2'].iloc[9]

'```python\nimport pandas as pd\n\ndata = {\n    \'title\': [\'Crime and Punishment\'],\n    \'author\': [\'Fyodor Dostoevsky\'],\n    \'female_major_character\': [1],\n    \'expl\': ["Sonya Marmeladova plays a significant role as a central character, contributing to Raskolnikov\'s moral transformation and serving as the first person he confesses to. Her presence and influence are crucial throughout the narrative, although Raskolnikov is the protagonist."]\n}\n\ndf = pd.DataFrame(data)\n\nprint(df)\n```'

Crime and punishment
the giver

overlap?

save name of fmc

Try in batch? 